<a href="https://colab.research.google.com/github/emmanuelagubata/NoHARMS-/blob/main/UpdatingDashBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#reorgainzed code
#working to get display working

E: Unable to locate package python3-novu
Found existing installation: novu 1.14.0
Uninstalling novu-1.14.0:
  Successfully uninstalled novu-1.14.0
  Using cached novu-1.14.0-py3-none-any.whl.metadata (9.8 kB)
Using cached novu-1.14.0-py3-none-any.whl (59 kB)


In [ ]:
## ----------------------------------------
#🔹 Step 1: Install Required Packages
# ----------------------------------------

# Install necessary dependencies
!pip install --upgrade pip setuptools wheel
!pip install ultralytics streamlit pandas openpyxl numpy opencv-python-headless smtplib email
!pip uninstall -y novu && pip install novu
!apt-get -qq install -y python3-novu && pip install novu
!pip install pyngrok



  Using cached ultralytics-8.3.90-py3-none-any.whl.metadata (35 kB)
  Using cached streamlit-1.43.2-py2.py3-none-any.whl.metadata (8.9 kB)
ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5; 8.0.10 Requires-Python >=3.7,<=3.11; 8.0.11 Requires-Python >=3.7,<=3.11; 8.0.12 Requires-Python >=3.7,<=3.11; 8.0.13 Requires-Python >=3.7,<=3.11; 8.0.14 Requires-Python >=3.7,<=3.11; 8.0.15 Requires-Python >=3.7,<=3.11; 8.0.16 Requires-Python >=3.7,<=3.11; 8.0.17 Requires-Python >=3.7,<=3.11; 8.0.18 Requires-Python >=3.7,<=3.11; 8.0.19 Requires-Python >=3.7,<=3.11; 8.0.20 Requires-Python >=3.7,<=3.11; 8.0.21 Requires-Python >=3.7,<=3.11; 8.0.22 Requires-Python >=3.7,<=3.11; 8.0.23 Requires-Python >=3.7,<=3.11; 8.0.24 Requires-Python >=3.7,<=3.11; 8.0.25 Requires-Python >=3.7,<=3.11; 8.0.26 Requires-Python >=3.7,<=3.11; 8.0.27 Requires-Python >=3.7,<=3.11; 8.0.28 Requires-Python >=3.7,<=3.11; 8.0.29 Requires-Python >=3.7,<=3.11; 8.0.30 Requir

In [ ]:
# ----------------------------------------
# 🔹 Step 2: Create & Run the Streamlit Dashboard (Final UI Fixes)
# ----------------------------------------
%%writefile app.py

import streamlit as st
import pandas as pd
import time

# Load patient data
patients = {
    "John Doe": {
        "Age": 72, "Gender": "Male", "Past Illness": "Diabetes",
        "Color": "#1E1E1E",
        "Tasks": {"Sleep (Lay in Bed)": [2, 3], "Use Bathroom": [3, 5], "Go to Kitchen": [1, 2]}
    },
    "Jane Smith": {
        "Age": 68, "Gender": "Female", "Past Illness": "Hypertension",
        "Color": "#3B82F6",
        "Tasks": {"Sleep (Lay in Bed)": [3, 4], "Use Bathroom": [2, 5], "Go to Kitchen": [1, 3]}
    },
    "Alex Johnson": {
        "Age": 75, "Gender": "Male", "Past Illness": "Arthritis",
        "Color": "#F59E0B",
        "Tasks": {"Sleep (Lay in Bed)": [1, 2], "Use Bathroom": [4, 6], "Go to Kitchen": [2, 3]}
    }
}

# Dashboard layout
st.set_page_config(page_title="Patient Activity Tracker", layout="wide")
st.title("🏥 Patient Activity Tracker")
st.write("Live tracking of patient movements.")

# Select patient
selected_patient = st.sidebar.selectbox("Select Patient", list(patients.keys()))
patient_data = patients[selected_patient]

# Load activity log dynamically
@st.cache_data
def load_activity_log():
    return pd.read_excel("/content/activity_log.xlsx")

st.subheader("🔄 Live Activity Log")
while True:
    df = load_activity_log()
    st.write(df)
    time.sleep(5)  # Refresh every 5 seconds


Overwriting app.py


In [ ]:
# ----------------------------------------
# 🔹 Step 3: Start Streamlit Server with ngrok
# ----------------------------------------
import os
from pyngrok import ngrok

# **Authenticate with ngrok (replace <your_authtoken> with your actual authtoken)**
!ngrok authtoken 2tn51aiwFfTMZEb1ZzUH6SJ6xmW_5PD7z86S4DuZzjfryNHCr


# Kill previous tunnels
!pkill -9 streamlit

# Run Streamlit App
!nohup streamlit run app.py --server.port 8501 &

# Expose Streamlit via ngrok
public_url = ngrok.connect(8501)
print(f"🌐 Access the app at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
nohup: appending output to 'nohup.out'
nohup: failed to run command 'streamlit': No such file or directory


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2uJyp4D7lw76kKAC8lI1afGVbHR, tn_2uJyu8EzSyczC8MUuhT8Z78fAzQ, tn_2uJzewIX9F8tzFEUSLDw7joOnSD\n\r\n\r\nERR_NGROK_324\r\n"}}


In [ ]:
# ----------------------------------------
# 🔹 Step 4: Process Video in Parallel & Update Dashboard (Using Novu)
# ----------------------------------------

import time
import requests
import pandas as pd
import cv2
import numpy as np
from datetime import datetime, timedelta
from ultralytics import YOLO
import smtplib
from email.message import EmailMessage

# Configuration
video_path = "/content/video1.mp4"
output_path = "/content/output_video.mp4"
excel_file = "/content/activity_log.xlsx"

# YOLO Model
model = YOLO("yolo11n.pt")
model.overrides['verbose'] = False

# Room Definitions
room_names = ["Living Room", "Kitchen", "Bedroom", "Bathroom"]
test_boxes = [(660, 700, 1220, 350), (0, 20, 1300, 300), (480, 380, 1430, 300), (0, 380, 470, 300)]

# Email Notification Function
def send_email_notification(event, room):
    sender_email = "your_email@example.com"
    receiver_email = "your_email@example.com"
    password = "your_email_password"

    msg = EmailMessage()
    msg["Subject"] = "🏥 Patient Activity Alert"
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg.set_content(f"Patient {event} {room}.")

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.send_message(msg)
        print(f"✅ Email sent: Patient {event} {room}.")
    except Exception as e:
        print(f"❌ Email failed: {e}")

# Process Video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = 0
previous_room = None
start_time = datetime.strptime("02:03:30", "%H:%M:%S")

activity_list = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    results = model.track(frame, persist=True)
    elapsed_seconds = frame_count / fps
    current_time = start_time + timedelta(seconds=elapsed_seconds)

    detected_room = None

    for idx, room_box in enumerate(test_boxes):
        room_x, room_y, room_w, room_h = room_box
        room_name = room_names[idx]

        person_in_room = any(
            box.cls == 0 and
            box.xyxy[0][0] < room_x + room_w and box.xyxy[0][2] > room_x and
            box.xyxy[0][1] < room_y + room_h and box.xyxy[0][3] > room_y
            for box in results[0].boxes
        )

        if person_in_room:
            detected_room = idx

    if detected_room is not None and detected_room != previous_room:
        if previous_room is not None:
            exit_room = room_names[previous_room]
            activity_list.append(f"🚪 Left {exit_room} at {current_time.strftime('%H:%M:%S')}")
            send_email_notification("exited", exit_room)

        enter_room = room_names[detected_room]
        activity_list.append(f"🏠 Entered {enter_room} at {current_time.strftime('%H:%M:%S')}")
        send_email_notification("entered", enter_room)

        previous_room = detected_room

    frame_count += 1

cap.release()

# Save Activity Log
df = pd.DataFrame(activity_list, columns=["Activity"])
df.to_excel(excel_file, index=False)
print(f"📁 Exported activity log to: {excel_file}")


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# ----------------------------------------
# 🔹 Step 5: Update Dashboard & Send Notifications in Real-Time
# ----------------------------------------
import streamlit as st
import pandas as pd
import time

@st.cache_data
def load_activity_log():
    return pd.read_excel("/content/activity_log.xlsx")

st.subheader("🔄 Live Activity Log")

while True:
    df = load_activity_log()
    st.write(df)
    time.sleep(5)  # Refresh every 5 seconds


2025-03-10 18:46:51.466 No runtime found, using MemoryCacheStorageManager
2025-03-10 18:46:51.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:46:51.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:46:51.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:46:51.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:46:51.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:46:51.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:46:51.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


FileNotFoundError: [Errno 2] No such file or directory: '/content/activity_log.xlsx'